In [1]:
import pandas as pd
import numpy as np

Importing the Data

In [3]:
df = pd.read_csv(r"C:\Users\Mohdd\Downloads\archive\train.csv")

In [86]:
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,per_month_price
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,private room,149,1,9,2018-10-19,0.21,6,365,4532.083333
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,entire home/apt,225,1,45,2019-05-21,0.38,2,355,6843.750000
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,private room,150,3,0,NaN,NaN,1,365,4562.500000
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,entire home/apt,89,1,270,2019-07-05,4.64,1,194,2707.083333
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,entire home/apt,80,10,9,2018-11-19,0.10,1,0,2433.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,private room,70,2,0,NaN,NaN,2,9,2129.166667
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,private room,40,4,0,NaN,NaN,2,36,1216.666667
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,entire home/apt,115,10,0,NaN,NaN,1,27,3497.916667
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,shared room,55,1,0,NaN,NaN,6,2,1672.916667


In [6]:
df.shape

(48895, 16)

Cleaning the price column

In [32]:
df['price']

0        149
1        225
2        150
3         89
4         80
        ... 
48890     70
48891     40
48892    115
48893     55
48894     90
Name: price, Length: 48895, dtype: int64

In [55]:
df["price"] = df["price"].astype(int)

In [63]:
zero_valued = df[df['price']==0]

In [65]:
df = df.drop(zero_valued.index)

looking for nan/null values

In [68]:
df['price'].isnull().sum()

0

Calculating average price

In [75]:
df['price'].describe()

count    48884.000000
mean       152.755053
std        240.170260
min         10.000000
25%         69.000000
50%        106.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64

In [72]:
df['price'].mean()

152.75505277800508

Comparing costs to the private rental market

In [76]:
df['per_month_price']= df['price']*365/12

In [80]:
round(df['per_month_price'].mean(),2)

4646.3


Cleaning the room type column

In [85]:
df['room_type']= df['room_type'].str.lower()

In [96]:
room_freq= df['room_type'].astype('category')
room_freq.value_counts()

room_type
entire home/apt    25407
private room       22319
shared room         1158
Name: count, dtype: int64

What timeframe are we working with?

In [98]:
df['last_review'] = pd.to_datetime(df['last_review'])

In [99]:
df['last_review'].max()

Timestamp('2019-07-08 00:00:00')

In [101]:
df['last_review'].min()

Timestamp('2011-03-28 00:00:00')

Analyzing listing prices by NYC borough

In [152]:
borough = df.groupby('neighbourhood_group')['price'].agg(['sum','mean','median','count'])
borough = borough.round(2).sort_values('mean',ascending=False)
borough

,sum,mean,median,count
neighbourhood_group,,,,
Manhattan,4264527,196.88,150.0,21660
Brooklyn,2500600,124.44,90.0,20095
Staten Island,42825,114.81,75.0,373
Queens,563867,99.52,75.0,5666
Bronx,95459,87.58,65.0,1090


Price range by borough

In [148]:
label = ['low','budget','averge','expensive']
ranges = [0, 69, 175, 350, np.inf]
df['price_range'] = pd.cut(df['price'], bins=ranges ,labels=label)

In [159]:
price_range_by_borough =  df.groupby(['price_range','neighbourhood_group'],observed=False)['price_range'].count()

In [160]:
price_range_by_borough 

price_range  neighbourhood_group
low          Bronx                    573
             Brooklyn                6563
             Manhattan               2513
             Queens                  2549
             Staten Island            162
budget       Bronx                    455
             Brooklyn               10274
             Manhattan              10872
             Queens                  2577
             Staten Island            169
averge       Bronx                     44
             Brooklyn                2713
             Manhattan               6470
             Queens                   460
             Staten Island             34
expensive    Bronx                     18
             Brooklyn                 545
             Manhattan               1805
             Queens                    80
             Staten Island              8
Name: price_range, dtype: int64